In [15]:
import datetime as dt
import pandas as pd
import numpy as np
import plotly.express as px
import yfinance as yf

Raw Data Gathering

In [16]:
effr_raw=pd.read_csv("https://markets.newyorkfed.org/api/rates/unsecured/effr/last/61.csv")
effr=effr_raw.copy()
effr['FF']=effr['Rate (%)']
effr['FF CoD']=effr['FF'].diff(periods=-1)*100


effr['FF to mid target']=(effr['FF']-((effr['Target Rate From (%)']+effr['Target Rate To (%)'])/2))*100
effr

,Effective Date,Rate Type,Rate (%),1st Percentile (%),25th Percentile (%),75th Percentile (%),99th Percentile (%),Volume ($Billions),Target Rate From (%),Target Rate To (%),...,Standard Deviation (%),30-Day Average SOFR,90-Day Average SOFR,180-Day Average SOFR,SOFR Index,Revision Indicator (Y/N),Footnote ID,FF,FF CoD,FF to mid target
0,10/02/2024,EFFR,4.83,4.81,4.83,4.84,4.85,81,4.75,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.83,0.0,-4.5
1,10/01/2024,EFFR,4.83,4.81,4.83,4.84,4.85,85,4.75,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.83,0.0,-4.5
2,09/30/2024,EFFR,4.83,4.80,4.82,4.84,4.95,73,4.75,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.83,0.0,-4.5
3,09/27/2024,EFFR,4.83,4.81,4.82,4.84,4.86,94,4.75,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.83,0.0,-4.5
4,09/26/2024,EFFR,4.83,4.81,4.82,4.84,4.85,100,4.75,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.83,0.0,-4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,07/15/2024,EFFR,5.33,5.31,5.32,5.34,5.45,88,5.25,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.33,0.0,-4.5
57,07/12/2024,EFFR,5.33,5.31,5.32,5.34,5.45,88,5.25,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.33,0.0,-4.5
58,07/11/2024,EFFR,5.33,5.31,5.32,5.34,5.45,90,5.25,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.33,0.0,-4.5
59,07/10/2024,EFFR,5.33,5.31,5.32,5.34,5.45,88,5.25,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.33,0.0,-4.5


In [17]:

effr=effr[['Effective Date','FF','FF CoD','FF to mid target']]

effr 

,Effective Date,FF,FF CoD,FF to mid target
0,10/02/2024,4.83,0.0,-4.5
1,10/01/2024,4.83,0.0,-4.5
2,09/30/2024,4.83,0.0,-4.5
3,09/27/2024,4.83,0.0,-4.5
4,09/26/2024,4.83,0.0,-4.5
...,...,...,...,...
56,07/15/2024,5.33,0.0,-4.5
57,07/12/2024,5.33,0.0,-4.5
58,07/11/2024,5.33,0.0,-4.5
59,07/10/2024,5.33,0.0,-4.5


In [18]:
sofr_raw=pd.read_csv('https://markets.newyorkfed.org/api/rates/secured/sofr/last/91.csv')
sofr=sofr_raw.copy()
sofr['SOFR']=sofr['Rate (%)']
sofr['SOFR CoD']=sofr['SOFR'].diff(periods=-1)*100
sofr=sofr[['Effective Date','SOFR','SOFR CoD']]

effr_sofr=pd.merge(effr,sofr,how='right')
effr_sofr=effr_sofr.rename(columns={'Effective Date':'Value Date'})
effr_sofr['Value Date']=pd.to_datetime(effr_sofr['Value Date'])
effr_sofr['Value Date']=effr_sofr['Value Date'].dt.strftime("%a %d%b%y")

effr_sofr_backfilled=effr_sofr[:90].copy()
effr_sofr_backfilled.fillna(method='bfill',inplace=True)
effr_sofr['FF - SOFR Spread']=(effr_sofr['FF']-effr_sofr['SOFR'])*100
effr_sofr=effr_sofr.fillna('')
effr_sofr 

C:\Users\danie\AppData\Local\Temp\ipykernel_28380\3639283593.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  effr_sofr_backfilled.fillna(method='bfill',inplace=True)


,Value Date,FF,FF CoD,FF to mid target,SOFR,SOFR CoD,FF - SOFR Spread
0,Wed 02Oct24,4.83,0.0,-4.5,4.92,-13.0,-9.0
1,Tue 01Oct24,4.83,0.0,-4.5,5.05,9.0,-22.0
2,Mon 30Sep24,4.83,0.0,-4.5,4.96,12.0,-13.0
3,Fri 27Sep24,4.83,0.0,-4.5,4.84,1.0,-1.0
4,Thu 26Sep24,4.83,0.0,-4.5,4.83,-1.0,0.0
...,...,...,...,...,...,...,...
86,Thu 30May24,,,,5.33,0.0,
87,Wed 29May24,,,,5.33,1.0,
88,Tue 28May24,,,,5.32,0.0,
89,Fri 24May24,,,,5.32,1.0,


In [19]:
# Get SOFR Futures (3 month) data from Yahoo Finance
ticker = 'SR3=F'

# Download the latest market data for the 3-month SOFR future
sofr_data = yf.Ticker(ticker)

In [20]:
used_ff_rate = effr_sofr['FF'][0]
used_ff_rate

4.83

In [28]:

# Get the current price (last closing price)
current_price = round(sofr_data.history(period="1d")['Close'].iloc[-1],3)

# Display the current price of the 3-month SOFR future
print(f"The latest 3-month SOFR future price is: {current_price}")
priced_in=round(100-current_price-used_ff_rate,3)
print(f"Amount of basis point cuts/hikes priced in is {priced_in}")
print(f"{priced_in//-.25} cuts are priced in with a {priced_in%.25/.25}% chance of another cut")

The latest 3-month SOFR future price is: 95.925
Amount of basis point cuts/hikes priced in is -0.755
3.0 cuts are priced in with a 0.98% chance of another cut
